In [20]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from lxml.etree import ParserError
import pandas as pd
import itertools
import numpy as np
from sklearn.metrics import classification_report



# Global Constansts
PATH = "C:\Program Files (x86)\chromedriver.exe"
BASE_URL = "http://en.wikipedia.org/wiki/"
HEADERS = {
    "Accept-Language": "en-US,en;q=0.9",
    "User-Agent": "Chrome/87.0.4280.141"
}
DELAY = 10

driver = webdriver.Chrome(executable_path=PATH)

name = "Agastache_foeniculum"
driver.get(BASE_URL + name)

try:
    WebDriverWait(driver, DELAY).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
    full_table_element = driver.find_element_by_class_name("mw-parser-output")
    df_list = pd.read_html(full_table_element.get_attribute("innerHTML"))
    data = df_list[0]

except TimeoutException:
    print("Time Error")

#Clean


driver.close()

In [57]:
classification_table = data.copy()

In [58]:
classification_table.iloc[:,0] = classification_table.iloc[:,0].str.strip(":")
classification_table = classification_table[classification_table["Agastache foeniculum"].isin(["Kingdom","Order","Family","Clade","Genus","Species"])]
plant_name = classification_table.columns[0]
classification_table.set_index(plant_name,inplace=True)
classification_table.columns = [plant_name]
classification_table

,Agastache foeniculum
Agastache foeniculum,
Kingdom,Plantae
Clade,Tracheophytes
Clade,Angiosperms
Clade,Eudicots
Clade,Asterids
Order,Lamiales
Family,Lamiaceae
Genus,Agastache
Species,A. foeniculum


In [42]:
number_of_clades = classification_table.index.value_counts()["Clade"]


In [59]:
i = 0
new_index = []
for index_value in classification_table.index:
    if index_value == "Clade":
        new_index.append(index_value + str(i))
        i+=1
    else:
        new_index.append(index_value)
classification_table.index = pd.Index(new_index)
classification_table


,Agastache foeniculum
Kingdom,Plantae
Clade0,Tracheophytes
Clade1,Angiosperms
Clade2,Eudicots
Clade3,Asterids
Order,Lamiales
Family,Lamiaceae
Genus,Agastache
Species,A. foeniculum


In [61]:
classification_table[plant_name] = classification_table[plant_name].apply(lambda x: x.replace(u'\xa0', u' '))
classification_table.to_dict()

{'Agastache foeniculum': {'Kingdom': 'Plantae',
  'Clade0': 'Tracheophytes',
  'Clade1': 'Angiosperms',
  'Clade2': 'Eudicots',
  'Clade3': 'Asterids',
  'Order': 'Lamiales',
  'Family': 'Lamiaceae',
  'Genus': 'Agastache',
  'Species': 'A. foeniculum'}}